In [1]:
import pandas as pd
import processing
from graphs import *

DATA_PATH = "./data/courses.json"

courses_df = pd.read_json(DATA_PATH, orient="index").dropna().filter(regex="infr.*", axis="index")

In [2]:
outline = courses_df["outline_level_year"]
start = courses_df["delivery_start"]

courses_df["year"] = pd.to_numeric(outline.apply(processing.year))
courses_df["start_semester"] = pd.to_numeric(start.apply(processing.semester))

courses_df["start_score"] = courses_df["year"] + courses_df["start_semester"] * 0.1
courses_df = courses_df.dropna()

prereqs = courses_df["entry_pre_req"].apply(processing.prereq)

In [3]:
G = PriorityGraph()

course_to_vertex = {}
vertex_to_course = {}

for course, _ in prereqs.items():
    i = G.addVertex(courses_df["start_score"][course])
    course_to_vertex[course] = i
    vertex_to_course[i] = course

for course, prereq in prereqs.items():
    for pre_course in prereq:
        if pre_course in course_to_vertex.keys():
            G.addEdge(course_to_vertex[pre_course], course_to_vertex[course])



In [4]:
L = G.topSort()
course_order = processing.list_order(map(lambda i : courses_df.loc[vertex_to_course[i]]["title"], L))
course_order

['Undergraduate Course: Informatics 1 - Introduction to Computation (INFR08025)',
 'Undergraduate Course: Informatics 1 - Cognitive Science (INFR08020)',
 'Undergraduate Course: Informatics 1 - Object Oriented Programming (INFR08029)',
 'Undergraduate Course: Informatics 2C - Introduction to Computer Systems (INFR08027)',
 'Undergraduate Course: Informatics Experiential Learning (Level 8) (INFR08033)',
 'Undergraduate Course: Informatics 2 - Foundations of Data Science (INFR08030)',
 'Undergraduate Course: Discrete Mathematics and Probability (INFR08031)',
 'Undergraduate Course: Informatics 2 - Introduction to Algorithms and Data Structures (INFR08026)',
 'Undergraduate Course: Informatics 2 - Software Engineering and Professional Practice (INFR08032)',
 'Undergraduate Course: Informatics 2D - Reasoning and Agents (INFR08010)',
 'Undergraduate Course: Elements of Programming Languages (INFR10061)',
 'Undergraduate Course: Informatics Large Practical (INFR09051)',
 'Undergraduate Cours

TODO: Connected components